In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
from matplotlib.font_manager import FontProperties
#font=FontProperties(fname='C:/Windows/Fonts/msyh.ttf')
%matplotlib inline
#導入ADF函數和numpy包
import warnings
warnings.filterwarnings('ignore')
from arch.unitroot import ADF
import numpy as np
import re
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import yfinance as yf
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] #中文錯誤問題

In [2]:
StockSymbol = ["1102.TW","1216.TW","1301.TW","1303.TW","1326.TW","1402.TW","2002.TW","2105.TW","2207.TW","2227.TW","2301.TW","2303.TW","2308.TW","2317.TW","2330.TW","2352.TW","2354.TW","2357.TW","2382.TW","2395.TW","2408.TW","2409.TW","2412.TW","2454.TW","2474.TW","2609.TW","2610.TW","2801.TW","2823.TW","2880.TW","2881.TW","2882.TW","2883.TW","2884.TW","2885.TW","2886.TW","2887.TW","2890.TW","2891.TW","2892.TW","2912.TW","3008.TW","3045.TW","3481.TW","3711.TW","4904.TW","4938.TW","5880.TW","6505.TW","9904.TW"]
StockName = ["TW1102","TW1216","TW1301","TW1303","TW1326","TW1402","TW2002","TW2105","TW2207","TW2227","TW2301","TW2303","TW2308","TW2317","TW2330","TW2352","TW2354","TW2357","TW2382","TW2395","TW2408","TW2409","TW2412","TW2454","TW2474","TW2609","TW2610","TW2801","TW2823","TW2880","TW2881","TW2882","TW2883","TW2884","TW2885","TW2886","TW2887","TW2890","TW2891","TW2892","TW2912","TW3008","TW3045","TW3481","TW3711","TW4904","TW4938","TW5880","TW6505","TW9904"]
StockDataList = ["TW1102","TW1216","TW1301","TW1303","TW1326","TW1402","TW2002","TW2105","TW2207","TW2227","TW2301","TW2303","TW2308","TW2317","TW2330","TW2352","TW2354","TW2357","TW2382","TW2395","TW2408","TW2409","TW2412","TW2454","TW2474","TW2609","TW2610","TW2801","TW2823","TW2880","TW2881","TW2882","TW2883","TW2884","TW2885","TW2886","TW2887","TW2890","TW2891","TW2892","TW2912","TW3008","TW3045","TW3481","TW3711","TW4904","TW4938","TW5880","TW6505","TW9904"]

In [3]:
#最小距離法
def SSD(priceX,priceY):
    if priceX is None or priceY is None:
        print('缺少價格序列.')
    standardX=priceX/priceX[0]
    standardY=priceY/priceY[0]
    SSD=np.sum((standardX-standardY)**2)
    return(SSD) 

In [4]:
def spreadCal(priceX,priceY):
    data = pd.concat([priceX,priceY],axis=1).dropna()
    data.columns=['X','Y']
    standardX=data.X/data.X[0]
    standardY=data.Y/data.Y[0]
    spread=standardX-standardY
    return(spread)

In [5]:
######PairTrading Class #############
class PairTrading:
    def SSD(self,priceX,priceY):
        if priceX is None or priceY is None:
            pass
        standardX=priceX/priceX[0]
        standardY=priceY/priceY[0]
        SSD=np.sum((standardY-standardX)**2)
        return(SSD)
    def SSDSpread(self,priceX,priceY):
        if priceX is None or priceY is None:
            pass
        standardX=priceX/priceX[0]
        standardY=priceY/priceY[0]
        spread=standardY-standardX
        return(spread)
    def cointegration(self,priceX,priceY):
        if priceX is None or priceY is None:
            print('缺少價格序列.')
        priceX=np.log(priceX)
        priceY=np.log(priceY)
        results=sm.OLS(priceY,sm.add_constant(priceX)).fit()
        resid=results.resid
        adfSpread=ADF(resid)
        if adfSpread.pvalue>=0.05:
            return(0,0,False)
        else:
            boolen = True
            print('''交易價格具有協整關係.
            P-value of ADF test: %f
            Coefficients of regression:
            Intercept: %f
            Beta: %f
             ''' % (adfSpread.pvalue, results.params[0], results.params[1]))
            return(results.params[0], results.params[1],boolen)
    def CointegrationSpread(self,priceX,priceY,
                            formStart,formEnd,tradeStart,tradeEnd):
        if priceX is None or priceY is None:
            pass
        formX=priceX[formStart:formEnd]
        formY=priceY[formStart:formEnd]
        tradeX=priceX[tradeStart:tradeEnd]
        tradeY=priceY[tradeStart:tradeEnd]
        coefficients=self.cointegration(formX,formY)
        if coefficients is None:
                pass
        else:
            spread=(np.log(tradeY)
            -coefficients[0]-coefficients[1]*np.log(tradeX))
            return(spread)
    def calBound(self,priceX,priceY,method,formStart,formEnd,width=1.5):
        formX=priceX[formStart:formEnd]
        formY=priceY[formStart:formEnd]
        if method=='SSD':
            spread=self.SSDSpread(formX,formY)            
            mu=np.mean(spread)
            sd=np.std(spread)
            UpperBound=mu+width*sd
            LowerBound=mu-width*sd
            return(UpperBound,LowerBound)
        elif method=='Cointegration':
            spread=self.CointegrationSpread(priceX,priceY,formStart,formEnd,
                 formStart,formEnd)
            mu=np.mean(spread)
            sd=np.std(spread)
            UpperBound=mu+width*sd
            LowerBound=mu-width*sd
            return(UpperBound,LowerBound)
        else:
            pass

In [6]:
#模擬交易
def TradeSig(prcLevel):
    n=len(prcLevel)
    signal=np.zeros(n)
    for i in range(1,n):
        if prcLevel[i-1]==1 and prcLevel[i]==2:
            signal[i]=-2
        elif prcLevel[i-1]==1 and prcLevel[i]==0:
            signal[i]=2
        elif prcLevel[i-1]==2 and prcLevel[i]==3:
            signal[i]=3
        elif prcLevel[i-1]==-1 and prcLevel[i]==-2:
            signal[i]=1
        elif prcLevel[i-1]==-1 and prcLevel[i]==0:
            signal[i]=-1
        elif prcLevel[i-1]==-2 and prcLevel[i]==-3:
            signal[i]=-3
    return(signal)

In [7]:
def TradeSim(priceX,priceY,position):
    n=len(position)
    shareY=10000*position
    shareX=[(-beta)*shareY[0]*priceY[0]/priceX[0]]
    cash=[10000]
    for i in range(1,n):
        shareX.append(shareX[i-1])
        cash.append(cash[i-1])
        if position[i-1]==0 and position[i]==1:
            shareX[i]=(-beta)*shareY[i]*priceY[i]/priceX[i]
            cash[i]=cash[i-1]-(shareY[i]*priceY[i]+shareX[i]*priceX[i])
        elif position[i-1]==0 and position[i]==-1:
            shareX[i]=(-beta)*shareY[i]*priceY[i]/priceX[i]
            cash[i]=cash[i-1]-(shareY[i]*priceY[i]+shareX[i]*priceX[i])
        elif position[i-1]==1 and position[i]==0:
            shareX[i]=0
            cash[i]=cash[i-1]+(shareY[i-1]*priceY[i]+shareX[i-1]*priceX[i])
        elif position[i-1]==-1 and position[i]==0:
            shareX[i]=0
            cash[i]=cash[i-1]+(shareY[i-1]*priceY[i]+shareX[i-1]*priceX[i])
    cash = pd.Series(cash,index=position.index)
    shareY=pd.Series(shareY,index=position.index)
    shareX=pd.Series(shareX,index=position.index)
    asset=cash+shareY*priceY+shareX*priceX
    account=pd.DataFrame({'Position':position,'ShareY':shareY,'ShareX':shareX,'Cash':cash,'Asset':asset})
    return(account)

---

<font size=6>**以下是50支股票所有可配對的所有可能性**</font>

In [ ]:
formStart='2020-01-01'
formEnd='2021-05-21'
txtarry = []
for i in range (len(StockSymbol)):
    formStart='2020-01-01'
    formEnd='2021-05-21'
    Stock1=yf.download(StockSymbol[i], start=formStart, end=formEnd ,group_by="ticker")
    Stock1['ROI']=(Stock1['Close'].shift(-1)-Stock1['Close'])/Stock1['Close']*100
    Stock1 = Stock1.dropna()
    Stock1.index=pd.to_datetime(Stock1.index)
    for j in range (i+1,len(StockSymbol)):
        formStart='2020-01-01'
        formEnd='2021-05-21'
        Stock2=yf.download(StockSymbol[j], start=formStart, end=formEnd ,group_by="ticker")
        Stock2['ROI']=(Stock2['Close'].shift(-1)-Stock2['Close'])/Stock2['Close']*100
        Stock2 = Stock2.dropna()
        Stock2.index=pd.to_datetime(Stock2.index)
        
        Stock1f=Stock1[formStart:formEnd]
        Stock2f=Stock2[formStart:formEnd]
        
        pairf=pd.concat([Stock1f.Close,Stock2f.Close],axis = 1)
        pairf.columns = [StockName[i],StockName[j]]
        
        Stock1f =  pairf[StockName[i]]  
        Stock2f = pairf[StockName[j]] 
        standardX=Stock1f/Stock1f[0]
        standardY=Stock2f/Stock2f[0]
        dis=np.sum((standardX-standardY)**2)
#         dis=SSD(Stock1f,Stock2f)
        
        Stock1flog=np.log(Stock1f)
        adfA=ADF(Stock1flog)
        
        retA=Stock1flog.diff()[1:]
        adfretA=ADF(retA)
        
        Stock2flog=np.log(Stock2f)
        adfB=ADF(Stock2flog)
        
        retB=Stock2flog.diff()[1:]
        adfretB=ADF(retB)
        
        Stock1flog = Stock1flog.dropna()
        Stock2flog = Stock2flog.dropna()
        
        model=sm.OLS(Stock1flog,sm.add_constant(Stock2flog))
        results=model.fit()
        
        alpha=results.params[0]  
        beta=results.params[1]

        spread=Stock1flog-beta*Stock2flog-alpha

        adfSpread=ADF(spread, trend='nc') 

        standardA= Stock1f/Stock1f[0]
        standardB= Stock2f/Stock2f[0]
        SSD_pair=standardA-standardB

        meanSSD_pair=np.mean(SSD_pair)
        sdSSD_pair=np.std(SSD_pair)
        thresholdUp=meanSSD_pair+1.5*sdSSD_pair
        thresholdDown=meanSSD_pair-1.5*sdSSD_pair
        
        tradStart='2021-01-01'
        tradEnd='2021-05-21'
        
        Stock1t=Stock1[tradStart:tradEnd].Close
        Stock2t=Stock1[tradStart:tradEnd].Close
        
        data = pd.concat([Stock1t,Stock2t],axis=1).dropna()
        data.columns=['X','Y']
        standardX=data.X/data.X[0]
        standardY=data.Y/data.Y[0]
        TradSpread=standardX-standardY
        

        spreadf=Stock1flog-beta*Stock2flog-alpha
        mu=np.mean(spreadf)
        sd=np.std(spreadf)

        CoSpreadT=np.log(Stock1t)-beta*np.log(Stock2t)-alpha
        
        formStart='2020-01-01'
        formEnd='2020-12-31'
        tradeStart='2021-01-01'
        tradeEnd='2021-05-21'

        priceA= Stock2.Close
        priceB=Stock1.Close
        priceAf=priceA[formStart:formEnd]
        priceBf=priceB[formStart:formEnd]
        priceAt=priceA[tradeStart:tradeEnd]
        priceBt=priceB[tradeStart:tradeEnd]

        pt=PairTrading()
        SSD=pt.SSD(priceAf,priceBf)

        SSDspread=pt.SSDSpread(priceAf,priceBf)
            
        print(str(StockName[i] + "與" + StockName[j] + "："))    
            
        coefficients=pt.cointegration(priceAf,priceBf)
        
        CoSpreadF=pt.CointegrationSpread(priceA,priceB,formStart,formEnd,formStart,formEnd)

        CoSpreadTr=pt.CointegrationSpread(priceA,priceB,formStart,formEnd,tradeStart,tradeEnd)
        
        bound=pt.calBound(priceA,priceB,'Cointegration',formStart,formEnd,width=1.2)
        
        if (coefficients[2] == True):
            mu=np.mean(CoSpreadF)
            sd=np.std(CoSpreadF)

            level=(float('-inf'),mu-2.5*sd,
                mu-1.5*sd,mu-0.2*sd,
                mu+0.2*sd,mu+1.5*sd,
                mu+2.5*sd,
                float('inf'))

            prcLevel=pd.cut(CoSpreadTr,level,labels=False)-3

            signal=TradeSig(prcLevel)

            position=[signal[0]]
            ns=len(signal)

            for g in range(1,ns):
                position.append(position[-1])
                if signal[g]==1:
                    position[g]=1
                elif signal[g]==-2:
                    position[g]=-1
                elif signal[g]==-1 and position[g-1]==1:
                    position[g]=0
                elif signal[g]==2 and position[g-1]==-1:
                    position[g]=0
                elif signal[g]==3:
                    position[g]=0
                elif signal[g]==-3:
                    position[g]=0

            position=pd.Series(position,index=CoSpreadT.index)

            account1=TradeSim(Stock2t,Stock1t,position)
            account1.tail() 
            print(str(StockName[i]) + "與" + str(StockName[j]) + "：" + str(account1.loc['2021-05-19','Asset']))
            txtarry.append(str(StockName[i]) + "與" + str(StockName[j]) + "：" + str(account1.loc['2021-05-19','Asset']))
        else:
            pass

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
TW1102與TW1216：
交易價格具有協整關係.
            P-value of ADF test: 0.004145
            Coefficients of regression:
            Intercept: 1.017743
            Beta: 0.649454
             
交易價格具有協整關係.
            P-value of ADF test: 0.004145
            Coefficients of regression:
            Intercept: 1.017743
            Beta: 0.649454
             
交易價格具有協整關係.
            P-value of ADF test: 0.004145
            Coefficients of regression:
            Intercept: 1.017743
            Beta: 0.649454
             
交易價格具有協整關係.
            P-value of ADF test: 0.004145
            Coefficients of regression:
            Intercept: 1.017743
            Beta: 0.649454
             
TW1102與TW1216：6365.091564676841
[*********************100%***********************]  1 of 1 completed
TW1102與TW1301：
交易價格具有協整關係.
            P-value of ADF test: 0.031565
         

[*********************100%***********************]  1 of 1 completed
TW1102與TW2352：
[*********************100%***********************]  1 of 1 completed
TW1102與TW2354：
交易價格具有協整關係.
            P-value of ADF test: 0.023800
            Coefficients of regression:
            Intercept: 1.164038
            Beta: 0.649978
             
交易價格具有協整關係.
            P-value of ADF test: 0.023800
            Coefficients of regression:
            Intercept: 1.164038
            Beta: 0.649978
             
交易價格具有協整關係.
            P-value of ADF test: 0.023800
            Coefficients of regression:
            Intercept: 1.164038
            Beta: 0.649978
             
交易價格具有協整關係.
            P-value of ADF test: 0.023800
            Coefficients of regression:
            Intercept: 1.164038
            Beta: 0.649978
             
TW1102與TW2354：24885.574403808045
[*********************100%***********************]  1 of 1 completed
TW1102與TW2357：
交易價格具有協整關係.
            P-value of ADF test: 0.

[*********************100%***********************]  1 of 1 completed
TW1102與TW2880：
交易價格具有協整關係.
            P-value of ADF test: 0.042859
            Coefficients of regression:
            Intercept: 1.760880
            Beta: 0.686228
             
交易價格具有協整關係.
            P-value of ADF test: 0.042859
            Coefficients of regression:
            Intercept: 1.760880
            Beta: 0.686228
             
交易價格具有協整關係.
            P-value of ADF test: 0.042859
            Coefficients of regression:
            Intercept: 1.760880
            Beta: 0.686228
             
交易價格具有協整關係.
            P-value of ADF test: 0.042859
            Coefficients of regression:
            Intercept: 1.760880
            Beta: 0.686228
             
TW1102與TW2880：5024.700832261762
[*********************100%***********************]  1 of 1 completed
TW1102與TW2881：
[*********************100%***********************]  1 of 1 completed
TW1102與TW2882：
交易價格具有協整關係.
            P-value of ADF test: 0.0

[*********************100%***********************]  1 of 1 completed
TW1102與TW3481：
交易價格具有協整關係.
            P-value of ADF test: 0.041110
            Coefficients of regression:
            Intercept: 3.668831
            Beta: 0.044814
             
交易價格具有協整關係.
            P-value of ADF test: 0.041110
            Coefficients of regression:
            Intercept: 3.668831
            Beta: 0.044814
             
交易價格具有協整關係.
            P-value of ADF test: 0.041110
            Coefficients of regression:
            Intercept: 3.668831
            Beta: 0.044814
             
交易價格具有協整關係.
            P-value of ADF test: 0.041110
            Coefficients of regression:
            Intercept: 3.668831
            Beta: 0.044814
             
TW1102與TW3481：7226.1586114435195
[*********************100%***********************]  1 of 1 completed
TW1102與TW3711：
[*********************100%***********************]  1 of 1 completed
TW1102與TW4904：
交易價格具有協整關係.
            P-value of ADF test: 0.

[*********************100%***********************]  1 of 1 completed
TW1216與TW2887：
[*********************100%***********************]  1 of 1 completed
TW1216與TW2890：
[*********************100%***********************]  1 of 1 completed
TW1216與TW2891：
[*********************100%***********************]  1 of 1 completed
TW1216與TW2892：
[*********************100%***********************]  1 of 1 completed
TW1216與TW2912：
[*********************100%***********************]  1 of 1 completed
TW1216與TW3008：
交易價格具有協整關係.
            P-value of ADF test: 0.002382
            Coefficients of regression:
            Intercept: 1.095277
            Beta: 0.379940
             
交易價格具有協整關係.
            P-value of ADF test: 0.002382
            Coefficients of regression:
            Intercept: 1.095277
            Beta: 0.379940
             
交易價格具有協整關係.
            P-value of ADF test: 0.002382
            Coefficients of regression:
            Intercept: 1.095277
            Beta: 0.379940
         

...

[*********************100%***********************]  1 of 1 completed
TW1303與TW2330：
[*********************100%***********************]  1 of 1 completed
TW1303與TW2352：
[*********************100%***********************]  1 of 1 completed
TW1303與TW2354：
[*********************100%***********************]  1 of 1 completed
TW1303與TW2357：
[*********************100%***********************]  1 of 1 completed
TW1303與TW2382：
[*********************100%***********************]  1 of 1 completed
TW1303與TW2395：
[*********************100%***********************]  1 of 1 completed
TW1303與TW2408：
交易價格具有協整關係.
            P-value of ADF test: 0.047618
            Coefficients of regression:
            Intercept: 2.624501
            Beta: 0.366300
             
交易價格具有協整關係.
            P-value of ADF test: 0.047618
            Coefficients of regression:
            Intercept: 2.624501
            Beta: 0.366300
             
交易價格具有協整關係.
            P-value of ADF test: 0.047618
            Coefficients

[*********************100%***********************]  1 of 1 completed
TW1326與TW2409：
[*********************100%***********************]  1 of 1 completed
TW1326與TW2412：
[*********************100%***********************]  1 of 1 completed
TW1326與TW2454：
[*********************100%***********************]  1 of 1 completed
TW1326與TW2474：
[*********************100%***********************]  1 of 1 completed
TW1326與TW2609：
[*********************100%***********************]  1 of 1 completed
TW1326與TW2610：
[*********************100%***********************]  1 of 1 completed
TW1326與TW2801：
[*********************100%***********************]  1 of 1 completed
TW1326與TW2823：
[*********************100%***********************]  1 of 1 completed
TW1326與TW2880：
[*********************100%***********************]  1 of 1 completed
TW1326與TW2881：
[*********************100%***********************]  1 of 1 completed
TW1326與TW2882：
[*********************100%***********************]  1 of 1 completed
TW1326與

[*********************100%***********************]  1 of 1 completed
TW1402與TW2352：
[*********************100%***********************]  1 of 1 completed
TW1402與TW2354：
[*********************100%***********************]  1 of 1 completed
TW1402與TW2357：
[*********************100%***********************]  1 of 1 completed
TW1402與TW2382：
交易價格具有協整關係.
            P-value of ADF test: 0.034561
            Coefficients of regression:
            Intercept: 2.300882
            Beta: 0.232755
             
交易價格具有協整關係.
            P-value of ADF test: 0.034561
            Coefficients of regression:
            Intercept: 2.300882
            Beta: 0.232755
             
交易價格具有協整關係.
            P-value of ADF test: 0.034561
            Coefficients of regression:
            Intercept: 2.300882
            Beta: 0.232755
             
交易價格具有協整關係.
            P-value of ADF test: 0.034561
            Coefficients of regression:
            Intercept: 2.300882
            Beta: 0.232755
          

[*********************100%***********************]  1 of 1 completed
TW1402與TW2884：
交易價格具有協整關係.
            P-value of ADF test: 0.024423
            Coefficients of regression:
            Intercept: 0.393422
            Beta: 0.894299
             
交易價格具有協整關係.
            P-value of ADF test: 0.024423
            Coefficients of regression:
            Intercept: 0.393422
            Beta: 0.894299
             
交易價格具有協整關係.
            P-value of ADF test: 0.024423
            Coefficients of regression:
            Intercept: 0.393422
            Beta: 0.894299
             
交易價格具有協整關係.
            P-value of ADF test: 0.024423
            Coefficients of regression:
            Intercept: 0.393422
            Beta: 0.894299
             
TW1402與TW2884：10466.102028533292
[*********************100%***********************]  1 of 1 completed
TW1402與TW2885：
[*********************100%***********************]  1 of 1 completed
TW1402與TW2886：
[*********************100%*********************

[*********************100%***********************]  1 of 1 completed
TW2002與TW2408：
[*********************100%***********************]  1 of 1 completed
TW2002與TW2409：
[*********************100%***********************]  1 of 1 completed
TW2002與TW2412：
[*********************100%***********************]  1 of 1 completed
TW2002與TW2454：
[*********************100%***********************]  1 of 1 completed
TW2002與TW2474：
[*********************100%***********************]  1 of 1 completed
TW2002與TW2609：
[*********************100%***********************]  1 of 1 completed
TW2002與TW2610：
[*********************100%***********************]  1 of 1 completed
TW2002與TW2801：
[*********************100%***********************]  1 of 1 completed
TW2002與TW2823：
[*********************100%***********************]  1 of 1 completed
TW2002與TW2880：
[*********************100%***********************]  1 of 1 completed
TW2002與TW2881：
[*********************100%***********************]  1 of 1 completed
TW2002與

In [ ]:
txtarry